## Accuracy Plots

In [14]:
from plotting import plot_classification_accuracies, plot_segmentation_accuracies
import pandas as pd
import altair_saver
import os

repbrep = '../../repbrep/'
outdir = 'figure_out'

segmentation_predictions = pd.read_parquet(os.path.join(repbrep, 'results/segmentation_predictions.parquet'))
classification_predictions = pd.read_parquet(os.path.join(repbrep, 'results/classification_predictions.parquet'))

f360_macro = plot_segmentation_accuracies(segmentation_predictions,  'Fusion360Seg', 'macro')
f360_macro_linear = plot_segmentation_accuracies(segmentation_predictions, 'Fusion360Seg', 'macro', 'linear')
f360_micro = plot_segmentation_accuracies(segmentation_predictions,  'Fusion360Seg', 'micro')
f360_micro_linear = plot_segmentation_accuracies(segmentation_predictions,  'Fusion360Seg', 'micro', 'linear')

mfcad_macro = plot_segmentation_accuracies(segmentation_predictions,  'MFCAD', 'macro')
mfcad_macro_linear = plot_segmentation_accuracies(segmentation_predictions,  'MFCAD', 'macro', 'linear')
mfcad_micro = plot_segmentation_accuracies(segmentation_predictions,  'MFCAD', 'micro')
mfcad_micro_linear = plot_segmentation_accuracies(segmentation_predictions,  'MFCAD', 'micro', 'linear')

fabwave = plot_classification_accuracies(classification_predictions, 'FabWave')
fabwave_linear = plot_classification_accuracies(classification_predictions, 'FabWave', 'Ours', 'linear')

os.makedirs(outdir, exist_ok=True)
altair_saver.save(f360_macro, os.path.join(outdir, 'f360_macro.pdf'))
altair_saver.save(f360_macro_linear, os.path.join(outdir, 'f360_macro_linear.pdf'))
altair_saver.save(mfcad_macro, os.path.join(outdir, 'mfcad_macro.pdf'))
altair_saver.save(mfcad_macro_linear, os.path.join(outdir, 'mfcad_macro_linear.pdf'))

altair_saver.save(f360_micro, os.path.join(outdir, 'f360_micro.pdf'))
altair_saver.save(f360_micro_linear, os.path.join(outdir, 'f360_micro_linear.pdf'))
altair_saver.save(mfcad_micro, os.path.join(outdir, 'mfcad_micro.pdf'))
altair_saver.save(mfcad_micro_linear, os.path.join(outdir, 'mfcad_micro_linear.pdf'))

altair_saver.save(fabwave, os.path.join(outdir, 'fabwave.pdf'))
altair_saver.save(fabwave_linear, os.path.join(outdir, 'fabwave.pdf'))

## Reconstruction Gallery

In [ ]:
import torch
import numpy as np
from rendering import find_best_angle_from_part, render_part, render_mesh
from tqdm.notebook import tqdm
from render_shape import preds_to_mesh
from zipfile import ZipFile
import json
from pspy import Part
from train_latent_space import BRepFaceAutoencoder
from matplotlib import pyplot as plt
import os

repbrep = '../../repbrep/'
datasets = os.path.join(repbrep, 'datasets')
model_dir = os.path.join(repbrep, 'models', 'BRepFaceAutoencoder_64_1024_4')
results_dir = os.path.join(repbrep, 'results')

f360seg_index_path = os.path.join(datasets, 'fusion360seg.json')
f360seg_zip_path = os.path.join(datasets, 'fusion360seg.zip')
model_checkpoint_path = os.path.join(model_dir, 'BRepFaceAutoencoder_64_1024_4.ckpt')
computed_f360seg_codes_path = os.path.join(model_dir, 'fusion360seg_coded.pt')
render_losses_path = os.path.join(results_dir, 'f360_render_test_losses.pt')
figure_out_path = os.path.join('figure_out', 'reconstructions.png')

test_poses_path = os.path.join(datasets, 'f360seg_test_poses.npy')
test_zooms_path = os.path.join(datasets, 'f360seg_test_zooms.npy')

rows = 6
cols = 4
size = 5

grid_density = 100

force_rendering = True



testing_losses = torch.load(render_losses_path)
avg_losses = np.array([x[0].item() for x in testing_losses])
part_sizes = np.array([x[1] for x in testing_losses])
total_losses = avg_losses * part_sizes
avg_sorted = sorted(enumerate(zip(avg_losses, part_sizes, total_losses)),key=lambda x: x[1][0])
total_sorted = sorted(enumerate(zip(avg_losses, part_sizes, total_losses)),key=lambda x: x[1][2])
total_filtered = [x for x in total_sorted if x[1][1] > 20]


recompute_renderings = True
if os.path.exists('render_results.pt') and not force_rendering:
    render_results = torch.load('render_results.pt')
    renders = render_results['renders']
    gts = render_results['gts']
    if len(renders) >= rows*cols:
        recompute_renderings = False

if recompute_renderings:
    with open(f360seg_index_path,'r') as f:
        index = json.load(f)
    data =  ZipFile(f360seg_zip_path,'r')
    parts_list = [index['template'].format(*x) for x in index['test']]

    model = BRepFaceAutoencoder(64,1024,4)
    ckpt = torch.load(model_checkpoint_path)
    model.load_state_dict(ckpt['state_dict'])

    def predict(face_codes, model, N=grid_density):
        n_faces = face_codes.shape[0]
        line = torch.linspace(-0.1,1.1,N)
        grid = torch.cartesian_prod(line, line)
        grids = grid.repeat(n_faces,1)
        indices = torch.arange(n_faces).repeat_interleave(N*N, dim=0)
        with torch.no_grad():
            indexed_codes = face_codes[indices]
            uv_codes = torch.cat([grids, indexed_codes],dim=1)
            preds = model.decoder(uv_codes)
        return preds

    codes = torch.load(computed_f360seg_codes_path)

    gts = []
    renders = []

    poses = np.load(test_poses_path)
    zooms = np.load(test_zooms_path)

    for k in tqdm(range(rows*cols)):
        i = total_filtered[k][0]
        N = grid_density
        preds = predict(codes[parts_list[i]]['x'], model, N)
        V, F, C = preds_to_mesh(preds, N)
        part = Part(data.open(parts_list[i]).read().decode('utf-8'))
        pose = poses[i]
        zoom = zooms[i]
        ground_truth = render_part(part, pose, zoom)
        rendering = render_mesh(V,F,C,pose,zoom)
        gts.append(ground_truth)
        renders.append(rendering)

    render_results = {'gts':gts,'renders':renders}
    torch.save(render_results, 'render_results.pt')



M = rows*cols
s = size
fig, axes = plt.subplots(int(M/cols), cols*2, figsize=(2*cols*s, s*int(M/cols)),gridspec_kw = {'wspace':0, 'hspace':0}, dpi=300)
for i in range(M):
    row = int(i / cols)
    col = i % cols
    axes[row,2*col].imshow(gts[i])
    axes[row,2*col].axis('off')
    axes[row,2*col+1].imshow(renders[i])
    axes[row,2*col+1].axis('off')
fig.savefig(figure_out_path)

## MFCAD Figure

In [ ]:
from precoded_training import DictDatamodule, CodePredictor, load_models
from pylab import cm
import meshplot as mp

with open('/projects/grail/benjonesnb/cadlab/siggasia2022/mfcad.json','r') as f:
    index = json.load(f)
    coded_set = torch.load('/projects/grail/benjonesnb/cadlab/siggasia2022/precoded/mfcad_coded.pt')

datamodule = DictDatamodule(
                    index, 
                    coded_set,  
                    seed=0, 
                    batch_size=512
                )

test_ds = datamodule.ds_test

all_models = load_models('/projects/grail/benjonesnb/cadlab/siggasia2022/tensorboard/mfcad/')
our_models = all_models['mp2']

# Compute a "difficulty" scale for test set examples (this is also used in comparison figs...)
accs = np.zeros((len(test_ds), len(our_models)))
for i in tqdm(range(len(test_ds))):
    data = test_ds[i]
    for j, (ts, m) in enumerate(our_models):
        with torch.no_grad():
            preds = m(data).argmax(dim=1).numpy()
        targets = data.y.numpy()
        acc = (preds.flatten() == targets.flatten()).sum() / len(targets.flatten())
        accs[i,j] = acc

# Setup Plotting Code (Replace with Other Version)
num_labels = max([i for j in index['test_labels'] for i in j]) + 1
cmap = cm.get_cmap('tab20', num_labels)
zf = ZipFile('/projects/grail/benjonesnb/cadlab/siggasia2022/mfcad.zip')
test_keys = [index['template'].format(*x) for x in index['test']]
def plot_part(V, F, E2T, FC=None):
    plot = mp.plot(V, F, c = FC)
    E = np.concatenate([
        F[E2T[:,0]>=0][:,[2,0]], 
        F[E2T[:,1]>=0][:,[0,1]],
        F[E2T[:,2]>=0][:,[1,2]]
    ], axis=0)
    plot.add_edges(V, E, shading={'line_width':0.5})
def plot_preds(i, test_ds, test_keys=test_keys, zf=zf, models=our_models, colormap=cmap, just_gt=False):
    data = test_ds[i]
    with zf.open(test_keys[i],'r') as f:
        p = Part(f.read().decode('utf-8'))
    target = data.y.numpy()
    target_c = colormap(target[p.mesh_topology.face_to_topology])[:,:3]
    plot_part(p.mesh.V, p.mesh.F, p.mesh_topology.edge_to_topology,  target_c)
    if just_gt:
        return
    model_preds = []
    for ts, model in models:
        with torch.no_grad():
            preds = model(data).argmax(dim=1).numpy()
            model_preds.append(preds)   
    for preds in model_preds:
        pred_c = colormap(preds[p.mesh_topology.face_to_topology])[:,:3]
        plot_part(p.mesh.V, p.mesh.F, p.mesh_topology.edge_to_topology, pred_c)


# Metrics used to help find the examples
local_auc = accs.sum(axis = 1) - accs[:,0] / 2 - accs[:,-1] / 2
complexity = np.array([len(test_ds[i].y) for i in range(len(test_ds))])
sorted_by_auc = sorted(enumerate(zip(local_auc, complexity)), key=lambda x:x[1][0])
filtered_examples = [x for x in sorted_by_auc if x[1][1] > 20]

gallery_examples = [646, 965, 673, 417, 888, 1090]

for i in gallery_examples:
    plot_preds(i, test_ds, test_keys, zf, our_models, cmap, just_gt=True)


## Limitations Plot

In [239]:
import altair as alt
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import altair_saver

accs = pd.read_parquet('df_face_acc.parquet')
accs['one'] = 1
iscorr = np.vectorize(lambda x: 'correct' if x == 1 else 'incorrect')
accs['correct'] = iscorr(accs.label)
accs['logmse'] = np.log(accs.mse)


alt.data_transformers.disable_max_rows()

lim_plot = alt.Chart(accs[accs.mse < 1]).mark_bar().encode(
    x=alt.X('logmse',bin=alt.Bin(maxbins=50)),
    y=alt.Y('sum(one)', stack='normalize', axis=alt.Axis(format='%')),
    color=alt.Color('label:N')
).facet(row='train_size')

altair_saver.save(lim_plot, 'figure_out/limitations.pdf')


## Classification Comparisons

In [ ]:
from plotting import render_segmentation_comparisons

render_segmentation_comparisons()

In [ ]:
from plotting import render_segmentation_comparisons


render_segmentation_comparisons(
    root='../../repbrep/',
    dataset_name='mfcad',
    camera_name='mfcad',
    dataset='MFCAD',
    max_labels=16
)

## Segmentation Gallery

In [9]:
import json
import meshplot as mp
from zipfile import ZipFile
from pspy import Part

with open('../../repbrep/datasets/fusion360seg.json', 'r') as f:
    index = json.load(f)

# Good examples: 
# 779, 3229, 2112, 345, 1709, 1067, 634, 2093, 2282, 535, 2947, 3494, 1979, 2995, 2616, 2052
# 27, 1290, 909
gallery_examples = [779, 1709, 1067, 634, 1979, 2052, 909]

with ZipFile('../../repbrep/datasets/fusion360seg.zip', 'r') as zf:
    parts = [Part(zf.open(index['template'].format(*p)).read().decode('utf-8')) for p in index['test']]
meshes = [p.mesh for p in parts]

In [20]:
from rendering import render_part2
import numpy as np
from matplotlib import pyplot as plt

In [17]:
poses = np.load('../../repbrep/datasets/f360seg_test_poses.npy')
zooms = np.load('../../repbrep/datasets/f360seg_test_zooms.npy')

In [62]:
old_plot_parts = [1,2,3,5,7]
images = [render_part2(parts[i], poses[i], zooms[i], viewport_width=200, viewport_height=200) for i in range(1000)]


In [57]:
n_cols = 10
n_rows = 20
tiled_images = np.zeros((200*n_rows,200*n_cols,3)).astype(int)
for row in range(n_rows):
    for col in range(n_cols):
        tiled_images[200*row:200*(row+1),200*col:200*(col+1),:] = images[10*row+col]

In [ ]:
fig,ax = plt.subplots(figsize=(n_rows*2,n_cols*2))
ax.imshow(tiled_images,aspect='equal')